In [6]:
import librosa
import librosa.display
import numpy as np
import soundfile as sf
import matplotlib.pyplot as plt
from pathlib import Path


class SimpleTechnoGenerator:
    """
    Simple Techno Pattern Generator for kick, snare, and hi-hat samples.
    Uses librosa for audio processing.
    """
    
    def __init__(self, 
                 kick_sound=None, snare_sound=None, hihat_sound=None,
                 bpm: int = 130,
                 sr: int = 44100):  # Lower SR for simplicity
        
        self.bpm = bpm
        self.sr = sr
        self.beat_duration = 60.0 / bpm  # seconds per beat
        self.bar_duration = self.beat_duration * 4  # 4 beats per bar
    
        self.kick = kick_sound
        self.snare =snare_sound
        self.hihat = hihat_sound
        
        print(f"Kick: {len(self.kick)/sr:.2f}s")
        print(f"Snare: {len(self.snare)/sr:.2f}s")
        print(f"Hi-hat: {len(self.hihat)/sr:.2f}s")
        
        # Calculate timing

        
    def create_pattern(self, 
                      bars: int = 4,
                      pattern_type: str = "four_on_floor",
                      complexity: float = 0.5) -> np.ndarray:
        """
        Create a techno drum pattern.
        
        Args:
            bars: Number of bars
            pattern_type: Type of pattern
            complexity: How complex the pattern is (0-1)
            
        Returns:
            Mono audio array
        """
        # Total duration in samples
        total_samples = int(self.bar_duration * bars * self.sr)
        
        # Initialize tracks
        kick_track = np.zeros(total_samples)
        snare_track = np.zeros(total_samples)
        hihat_track = np.zeros(total_samples)
        
        # Pattern definitions (16th note grid: 0-15 per bar)
        if pattern_type == "four_on_floor":
            # Classic techno: kick on every quarter
            kick_positions = [0, 4, 8, 12]
            snare_positions = [4, 12]  # on beats 2 and 4
            hihat_positions = list(range(0, 16, 2))  # 8th notes
            
        elif pattern_type == "syncopated":
            # More syncopated pattern
            kick_positions = [0, 3, 6, 10, 12, 14]
            snare_positions = [4, 12]
            hihat_positions = list(range(0, 16, 1))  # 16th notes (every position)
            
        elif pattern_type == "minimal":
            # Minimal techno
            kick_positions = [0, 8]
            snare_positions = [12]
            hihat_positions = [0, 4, 8, 12]
            
        elif pattern_type == "triplet":
            # Triplet feel
            kick_positions = [0, 8]
            snare_positions = [4, 12]
            hihat_positions = [0, 3, 6, 9, 12, 15]  # Triplets
            
        else:
            raise ValueError(f"Unknown pattern type: {pattern_type}")
        
        # 16th note duration in samples
        sixteenth_samples = int((self.bar_duration / 16) * self.sr)
        
        # Place hits for each bar
        for bar in range(bars):
            bar_start = bar * 16 * sixteenth_samples
            
            # Place kicks
            for pos in kick_positions:
                sample_pos = bar_start + (pos * sixteenth_samples)
                if sample_pos < total_samples:
                    # Add some velocity variation
                    velocity = 1.0 - (complexity * np.random.random() * 0.3)
                    self._place_sound(kick_track, sample_pos, self.kick * velocity)
            
            # Place snares
            for pos in snare_positions:
                sample_pos = bar_start + (pos * sixteenth_samples)
                if sample_pos < total_samples:
                    velocity = 0.9 - (complexity * np.random.random() * 0.2)
                    self._place_sound(snare_track, sample_pos, self.snare * velocity)
            
            # Place hi-hats with some randomness
            for pos in hihat_positions:
                sample_pos = bar_start + (pos * sixteenth_samples)
                if sample_pos < total_samples:
                    # Skip some hi-hats randomly based on complexity
                    if np.random.random() > complexity * 0.3:
                        velocity = 0.7 - (np.random.random() * 0.3)
                        self._place_sound(hihat_track, sample_pos, self.hihat * velocity)
        
        # Add extra complexity (ghost notes, variations)
        if complexity > 0.3:
            kick_track, snare_track, hihat_track = self._add_complexity(
                kick_track, snare_track, hihat_track, 
                bars, sixteenth_samples, complexity
            )
        
        # Combine tracks
        mono = kick_track + snare_track + hihat_track

        
        return mono
    
    def _place_sound(self, track: np.ndarray, start_sample: int, sound: np.ndarray):
        """Place a sound sample in a track."""
        end_sample = min(start_sample + len(sound), len(track))
        sound_length = end_sample - start_sample
        track[start_sample:end_sample] += sound[:sound_length]
    
    def _add_complexity(self, kick_track, snare_track, hihat_track, 
                       bars, sixteenth_samples, complexity):
        """Add complexity to the pattern."""
        total_samples = len(kick_track)
        
        # Add ghost snares (soft snares)
        for bar in range(bars):
            if np.random.random() < complexity * 0.5:
                # Add 1-3 ghost snares per bar
                num_ghosts = np.random.randint(1, 4)
                for _ in range(num_ghosts):
                    pos = np.random.randint(0, 16)
                    if pos not in [4, 12]:  # Not on main snares
                        sample_pos = bar * 16 * sixteenth_samples + pos * sixteenth_samples
                        if sample_pos < total_samples:
                            velocity = np.random.uniform(0.1, 0.3)
                            self._place_sound(snare_track, sample_pos, self.snare * velocity)
        
        # Add extra hi-hats
        if complexity > 0.6:
            for bar in range(bars):
                # Add some off-beat hi-hats
                for pos in [1, 3, 5, 7, 9, 11, 13, 15]:
                    if np.random.random() < complexity * 0.4:
                        sample_pos = bar * 16 * sixteenth_samples + pos * sixteenth_samples
                        if sample_pos < total_samples:
                            velocity = np.random.uniform(0.4, 0.6)
                            self._place_sound(hihat_track, sample_pos, self.hihat * velocity)
        
        return kick_track, snare_track, hihat_track
    
    def create_stereo_pattern(self, **kwargs) -> np.ndarray:
        """Create pattern with stereo width."""
        mono = self.create_pattern(**kwargs)
        
        # Create stereo version with simple widening
        stereo = np.zeros((len(mono), 2))
        
        # Simple stereo: slight delay on right channel
        delay_samples = int(0.005 * self.sr)  # 5ms delay
        
        stereo[:, 0] = mono
        if delay_samples < len(mono):
            stereo[delay_samples:, 1] = mono[:-delay_samples] * 0.8
        
        return stereo
    
    def export_pattern(self, 
                      output_path: str = "techno_pattern.wav",
                      pattern_type: str = "four_on_floor",
                      bars: int = 8,
                      complexity: float = 0.6):
        """Generate and export a pattern."""
        print(f"Generating {pattern_type} pattern...")
        
        stereo = self.create_stereo_pattern(
            pattern_type=pattern_type,
            bars=bars,
            complexity=complexity
        )
        
        sf.write(output_path, stereo, self.sr)
        print(f"Exported to {output_path}")
        
        return stereo
    

    
    def create_variations(self, 
                         base_pattern: str = "four_on_floor",
                         num_variations: int = 4,
                         bars: int = 4):
        """Create multiple variations of a pattern."""
        variations = []
        
        for i in range(num_variations):
            complexity = np.random.uniform(0.3, 0.8)
            
            if i == 0:
                pattern_type = base_pattern
            else:
                # Mix patterns for variations
                pattern_types = ["four_on_floor", "syncopated", "minimal", "triplet"]
                pattern_type = np.random.choice(pattern_types)
            
            audio = self.create_stereo_pattern(
                pattern_type=pattern_type,
                bars=bars,
                complexity=complexity
            )
            
            variations.append({
                "pattern": pattern_type,
                "complexity": complexity,
                "audio": audio
            })
            
            print(f"Variation {i+1}: {pattern_type} (complexity: {complexity:.2f})")
        
        return variations
    
    def create_loop_with_fill(self, 
                            pattern_type: str = "four_on_floor",
                            loop_bars: int = 7,
                            fill_bars: int = 1):
        """Create a loop with a fill on the last bar."""
        total_bars = loop_bars + fill_bars
        
        # Create main loop
        loop = self.create_pattern(
            pattern_type=pattern_type,
            bars=loop_bars,
            complexity=0.5
        )
        
        # Create fill pattern (more complex)
        fill = self.create_pattern(
            pattern_type="syncopated",  # Different pattern for fill
            bars=fill_bars,
            complexity=0.8
        )
        
        # Combine
        combined = np.concatenate([loop, fill])
        
        # Add crash on last beat of fill
        if fill_bars > 0:
            # Simple crash simulation (hi-hat with different characteristics)
            crash_samples = int(0.5 * self.sr)  # 0.5 second crash
            t = np.linspace(0, 0.5, crash_samples)
            crash = np.sin(2 * np.pi * 300 * t) * np.exp(-t / 0.3)
            crash += np.random.normal(0, 0.3, crash_samples) * 0.5
            
            # Place at end
            crash_start = len(combined) - int(self.beat_duration * self.sr)
            self._place_sound(combined, crash_start, crash)
        
        # Create stereo
        stereo = np.zeros((len(combined), 2))
        stereo[:, 0] = combined
        stereo[:, 1] = combined * 0.9  # Simple stereo
        
        
        return stereo


In [8]:
# Initialize
kick_wav_path = 'sound/test_kick_100.wav'
snare_wav_path = 'sound/shhhhhh1.wav'
hihat_wav_path = 'sound/pulse.wav'

complexity = 4     #0-10
bpm = 170
pattern_type = "four_on_floor"


out_dir = "py_out_techno"


kick = librosa.load(kick_wav_path)[0]
snare = librosa.load(snare_wav_path)[0]
hihat = librosa.load(hihat_wav_path)[0]




generator = SimpleTechnoGenerator(bpm=bpm, kick_sound=kick,snare_sound=snare,hihat_sound=hihat)

# 1. Create and export a basic pattern
audio = generator.export_pattern(
    output_path=f"{out_dir}/basic_techno.wav",
    pattern_type=pattern_type,
    bars=8,
    complexity=0.6
)

# 3. Create multiple variations
variations = generator.create_variations(
    base_pattern=pattern_type,
    num_variations=4,
    bars=4
)
# Export variations
for i, variation in enumerate(variations):
    sf.write(f"{out_dir}/variation_{i+1}.wav", variation["audio"], generator.sr)

# 4. Create a loop with fill
loop_with_fill = generator.create_loop_with_fill(
    pattern_type=pattern_type,
    loop_bars=7,
    fill_bars=1
)
sf.write(f"{out_dir}/techno_loop_with_fill.wav", loop_with_fill, generator.sr)

print("\nAll patterns generated successfully!")

Kick: 0.14s
Snare: 0.05s
Hi-hat: 0.05s
Generating four_on_floor pattern...
Exported to py_out_techno/basic_techno.wav
Variation 1: four_on_floor (complexity: 0.53)
Variation 2: four_on_floor (complexity: 0.35)
Variation 3: triplet (complexity: 0.59)
Variation 4: four_on_floor (complexity: 0.59)

All patterns generated successfully!
